In [27]:
import torch
import pandas as pd
import numpy as np

In [37]:
# Node features: shape [num_nodes, feature_dim]
X = torch.tensor([
    [1, 0],   # Node 0
    [0, 1],   # Node 1
    [1, 1],   # Node 2
    [0, 0]    # Node 3
], dtype=torch.float32)

# Adjacency matrix (with self-loops - A+I)
A = torch.tensor([
    [1, 1, 0, 0],
    [1, 1, 1, 0],
    [0, 1, 1, 1],
    [0, 0, 1, 1]
], dtype=torch.float32)

labels = torch.tensor([0, 1, 0, 1])  # e.g., binary classification

The equation defines the forward propagation rule for a **Graph Convolutional Network (GCN) layer**:

$$
H^{(l+1)} = \sigma \left( \tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)} \right)
$$

### Components:
1. **$\tilde{A} = A + I$**  
   - Adjacency matrix ($A$) with added self-connections (identity matrix $I$).  
2. **$\tilde{D}$**  
   - Degree matrix of $\tilde{A}$, where $\tilde{D}_{ii} = \sum_j \tilde{A}_{ij}$.  
3. **$\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2}$**  
   - Symmetric normalization of $\tilde{A}$ to prevent scaling issues.  
4. **$H^{(l)}$**  
   - Node feature matrix at layer $l$ (input features for the current layer).  
5. **$W^{(l)}$**  
   - Trainable weight matrix for layer $l$.  
6. **$\sigma(\cdot)$**  
   - Non-linear activation function (e.g., ReLU, Sigmoid).  

### Interpretation:
- The term $\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2}$ aggregates **normalized neighborhood features**.  
- $H^{(l)} W^{(l)}$ applies a learnable linear transformation to the input features.  
- The output $H^{(l+1)}$ is the activated node embeddings for the next layer.  


In [44]:
def normalize_adjacency(A):
    D = torch.diag(torch.sum(A, dim=1))
    D_inv_sqrt = torch.inverse(torch.sqrt(D))
    A_hat = D_inv_sqrt @ A @ D_inv_sqrt
    return A_hat

In [55]:
import torch.nn as nn
import torch.nn.functional as F

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        
    def forward(self, X, A_hat):
        result = A_hat@X@self.weight
#         print(f'Result = {result}')
#         print(f'Relu Result = {F.relu(result)}')
        return F.relu(result)
    

In [56]:
class GCN(nn.Module):
    def __init__(self, in_features, hidden_dim, out_features):
        super().__init__()
        self.gcn1 = GCNLayer(in_features, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, out_features)
        
    def forward(self, X, A_hat):
        X = self.gcn1(X, A_hat)
        X = self.gcn2(X, A_hat)
        return X

In [57]:
A_hat = normalize_adjacency(A)
model = GCN(in_features=2, hidden_dim=4, out_features=2)
output = model(X, A_hat)
print("Output Node Embeddings:\n", output)

Output Node Embeddings:
 tensor([[0.0000e+00, 0.0000e+00],
        [0.0000e+00, 8.0825e-05],
        [0.0000e+00, 8.0825e-05],
        [0.0000e+00, 9.8990e-05]], grad_fn=<ReluBackward0>)


## Classification

In [62]:
print(f"Node Labels: {labels}")
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training step
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    output = model(X, A_hat)
    loss = loss_function(output, labels)
    loss.backward()
    optimizer.step()
    if epoch%10 ==0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        

        
    
    
    

Node Labels: tensor([0, 1, 0, 1])
Epoch 0, Loss: 0.6640
Epoch 10, Loss: 0.6638
Epoch 20, Loss: 0.6636
Epoch 30, Loss: 0.6633
Epoch 40, Loss: 0.6631
Epoch 50, Loss: 0.6629
Epoch 60, Loss: 0.6627
Epoch 70, Loss: 0.6625
Epoch 80, Loss: 0.6622
Epoch 90, Loss: 0.6620


In [2]:
import torch
# Adjacency matrix (with self-loops)
A = torch.tensor([
    [1, 1, 0, 0],
    [1, 1, 1, 0],
    [0, 1, 1, 1],
    [0, 0, 1, 1]
], dtype=torch.float32)
A

tensor([[1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 1.],
        [0., 0., 1., 1.]])

In [3]:
A1 = torch.randn((4,4), dtype = torch.float32)
A1

tensor([[ 0.0895,  0.9148,  1.5258,  1.0418],
        [ 0.4404,  1.0214, -0.0601, -2.5915],
        [ 1.8097,  1.2275, -2.6075, -1.5441],
        [-0.8819,  1.1146, -1.2569,  2.0856]])

In [4]:
A2 = torch.eye(4)
A2

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [5]:
A1+A2

tensor([[ 1.0895,  0.9148,  1.5258,  1.0418],
        [ 0.4404,  2.0214, -0.0601, -2.5915],
        [ 1.8097,  1.2275, -1.6075, -1.5441],
        [-0.8819,  1.1146, -1.2569,  3.0856]])

In [24]:
degree = torch.sum(A, dim=1)
print(type(degree))
print(degree.shape)
degree

<class 'torch.Tensor'>
torch.Size([4])


tensor([2., 3., 3., 2.])

In [25]:
torch.diag(1.0 / torch.sqrt(degree))

tensor([[0.7071, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5774, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5774, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.7071]])

In [15]:
A1

tensor([[ 0.0895,  0.9148,  1.5258,  1.0418],
        [ 0.4404,  1.0214, -0.0601, -2.5915],
        [ 1.8097,  1.2275, -2.6075, -1.5441],
        [-0.8819,  1.1146, -1.2569,  2.0856]])

In [20]:
print(degree.unsqueeze(1).shape)
degree.unsqueeze(1)

torch.Size([4, 1])


tensor([[ 3.5719],
        [-1.1899],
        [-1.1144],
        [ 1.0613]])

In [21]:
A1.unsqueeze(1)

tensor([[[ 0.0895,  0.9148,  1.5258,  1.0418]],

        [[ 0.4404,  1.0214, -0.0601, -2.5915]],

        [[ 1.8097,  1.2275, -2.6075, -1.5441]],

        [[-0.8819,  1.1146, -1.2569,  2.0856]]])

In [26]:
torch.FloatTensor(5, 6)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [69]:
a_list = [[1, 2, 3], [2, 3, 4], [5, 45, 6], [42, 24, 5]]
tensors = [torch.tensor(x, dtype=torch.float32) for x in a_list]
stacked_tensor = torch.stack(tensors, dim=1)

In [70]:
stacked_tensor

tensor([[ 1.,  2.,  5., 42.],
        [ 2.,  3., 45., 24.],
        [ 3.,  4.,  6.,  5.]])